# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)
print(type(map_box_api))

# Import Data

In [ ]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("../../week6_pyviz/unit_6_pythonic_monopoly/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("../../week6_pyviz/unit_6_pythonic_monopoly/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

file_path = Path("../../week6_pyviz/unit_6_pythonic_monopoly/sfo_data_mean_df.csv")
sfo_mean_df = pd.read_csv(file_path, index_col="year")

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [26]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    sfo_data_mean = sfo_data.groupby("year")['housing_units'].mean()
    fig_housing_units = plt.figure() 
    plot_housing_units = sfo_data_mean.plot.bar().set_title("Housing Units in San Francisco from 2010 to 2016")
    max1 = sfo_data_mean.max()
    min1 = sfo_data_mean.min()
    std1 = sfo_data_mean.std()
    results = sfo_data_mean.plot.bar(ylim=(min1-std1, max1+std1), title="Housing Units in San Francisco from 2010 to 2016")
    plt.close(fig_housing_units)
    return pn.pane.Matplotlib(fig_housing_units, tight=True)

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    sfo_data_mean = sfo_data.groupby("year")['housing_units'].mean()
    fig_housing_units = plt.figure() 
    plot_housing_units = sfo_data_mean.plot.bar().set_title("Housing Units in San Francisco from 2010 to 2016")
    max1 = sfo_data_mean.max()
    min1 = sfo_data_mean.min()
    std1 = sfo_data_mean.std()
    results = sfo_data_mean.plot.bar(ylim=(min1-std1, max1+std1), title="Housing Units in San Francisco from 2010 to 2016")
    plt.close(fig_housing_units)
    return pn.pane.Matplotlib(fig_housing_units, tight=True)


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    fig_housing_units = plt.figure() 
    sfo_data_mean = sfo_data.groupby('year')['sale_price_sqr_foot', 'gross_rent'].mean()

    sfo_data_mean = sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby('year').mean()
    
    fig_housing_units = plt.figure()
    sfo_rent_data_plot = sfo_data_mean['gross_rent'].plot(title="Average Rent per Year in San Francisco")
    plt.close(fig_housing_units)
    return pn.pane.Matplotlib(fig_housing_units, tight=True)



def average_sales_price():
    """Average Sales Price Per Year."""
    
    fig_housing_units = plt.figure() 
    sfo_data_mean = sfo_data.groupby('year')['sale_price_sqr_foot', 'gross_rent'].mean()

    sfo_data_mean = sfo_data[['sale_price_sqr_foot', 'gross_rent']].groupby('year').mean()
    sfo_slaes_data_plot = sfo_data_mean['sale_price_sqr_foot'].plot(title="Average Sale Price per Square Foot in San Francisco")
    plt.close(fig_housing_units)
    return pn.pane.Matplotlib(fig_housing_units, tight=True)



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    average_price_by_neighborhood = sfo_data.groupby(["year", "neighborhood"]).mean()
    return average_price_by_neighborhood.hvplot.line("year", "sale_price_sqr_foot", groupby="neighborhood")



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    fig_housing_units = plt.figure() 
    neighborhood_prices = sfo_data.groupby(['neighborhood']).mean()
    top_ten_expensive = neighborhood_prices.sort_values(by='sale_price_sqr_foot', ascending = False).head(10)
    top_ten_expensive.reset_index(inplace=True)
    top_ten =  top_ten_expensive.plot.bar("neighborhood", "sale_price_sqr_foot")
    plt.close(fig_housing_units)
    return pn.pane.Matplotlib(fig_housing_units, tight=True)


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""  

    fig_housing_units = plt.figure() 
    neighborhood_prices = sfo_data.groupby(['neighborhood', 'gross_rent']).mean()
    most_expensive_neighborhoods_rent_sales = neighborhood_prices.sort_values(by='sale_price_sqr_foot', ascending = False).head(10)

    plt.close(fig_housing_units)
    return pn.pane.Matplotlib(fig_housing_units, tight=True)

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    neighborhood_prices = sfo_data.groupby(['neighborhood']).mean()
    top_ten_expensive = neighborhood_prices.sort_values(by='sale_price_sqr_foot', ascending = False).head(10)
    top_ten_expensive.reset_index(inplace=True)
    parallel = px.parallel_coordinates(top_ten_expensive,color="sale_price_sqr_foot")
    return parallel



def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""

    px.set_mapbox_access_token(map_box_api)
    neighborhood_prices = sfo_data.groupby(['neighborhood']).mean()
    neighborhood_prices = neighborhood_prices.reset_index()
    neighborhood_prices = neighborhood_prices.rename(columns = {"neighborhood": "Neighborhood"}) 
    df_join = pd.merge (neighborhood_prices,df_neighborhood_locations, on = "Neighborhood")
    scatter = px.scatter_mapbox(df_join, lat ="Lat", lon ="Lon", color = "gross_rent", size = "sale_price_sqr_foot",zoom=11, size_max = 16)
    plotly_panel = pn.pane.Plotly(scatter)
    plotly_panel._updates = True
    return plotly_panel


def sunburst():
    """Sunburst Plot."""
    
   ## df=px.data.gapminder().query("year ==")
    #fig =px.sunburst(df, path=["",""]), values="pop",
               # color ="green", hover_data=["iso_alpha"])
    #fig.show()            


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!


# Create a tab layout for the dashboard
# YOUR CODE HERE!


# Create the dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()
fig.show()